# 03. LangChain 모델 직렬화(Serialization) 가이드

이 노트북에서는 LangChain 모델과 체인을 저장하고 불러오는 방법을 배웁니다.

## 목차
1. 환경 설정 및 LangSmith 연결
2. 직렬화 가능성 확인 (`is_lc_serializable`)
3. JSON 직렬화 (`dumps`, `dumpd`)
4. Pickle 직렬화
5. 모델 저장 및 불러오기 실습
6. 체인 직렬화 실습

## 개요

모델 직렬화는 다음과 같은 상황에서 유용합니다:
- **모델 배포**: 훈련된 모델을 프로덕션 환경에 배포
- **캐싱**: 복잡한 체인 구성을 저장하여 재사용
- **버전 관리**: 모델의 다양한 버전을 관리
- **공유**: 팀원 간 모델 설정 공유

## 직렬화 방법 비교

| 방법 | 장점 | 단점 | 용도 |
|------|------|------|------|
| JSON (`dumps`) | 가독성 좋음, 플랫폼 독립적 | 일부 객체 직렬화 불가 | 설정 공유, 디버깅 |
| Pickle | 모든 Python 객체 지원 | Python 전용, 보안 위험 | 완전한 객체 저장 |

## 1. 환경 설정 및 LangSmith 연결

먼저 필요한 패키지를 임포트하고 환경 변수를 설정합니다.

In [ ]:
import os
import json
import pickle
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.load import dumps, dumpd, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

# is_lc_serializable 함수 임포트 (최신 버전 대응)
try:
    from langchain_core.load import dumpd as _test_dumpd
    def is_lc_serializable(obj):
        """객체가 LangChain 직렬화 가능한지 확인하는 함수"""
        try:
            _test_dumpd(obj)
            return True
        except Exception:
            return False
except ImportError:
    # 대체 구현
    def is_lc_serializable(obj):
        """기본 구현: dumpd로 테스트"""
        try:
            from langchain_core.load import dumpd
            dumpd(obj)
            return True
        except Exception:
            return False

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수 확인
print("🔍 환경 변수 확인 중...")

# Google API 키 확인
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("❌ GOOGLE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
print("✅ Google API Key: 설정됨")

# LangSmith API 키 확인 (선택사항)
langsmith_api_key = os.getenv("LANGCHAIN_API_KEY")
if langsmith_api_key:
    print("✅ LangSmith API Key: 설정됨")
    # LangSmith 설정
    os.environ["LANGCHAIN_TRACING_V2"] = "true"  # LangSmith 추적 활성화
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "model-serialization-demo"  # 프로젝트 이름 설정
    
    print(f"📊 LangSmith 프로젝트: {os.environ['LANGCHAIN_PROJECT']}")
    print(f"🔄 LangSmith 추적 활성화: {os.environ['LANGCHAIN_TRACING_V2']}")
    print("🌐 LangSmith 대시보드: https://smith.langchain.com/")
else:
    print("⚠️  LangSmith API Key가 설정되지 않았습니다.")
    print("   LangSmith 추적 없이 계속 진행합니다.")

print("\n✅ 환경 설정 완료!")

In [ ]:
# Gemini 모델 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_output_tokens=200
)

print("🤖 Gemini 모델 초기화 완료!")
print(f"   모델: {llm.model}")
print(f"   온도: {llm.temperature}")
print(f"   최대 토큰: {llm.max_output_tokens}")

# 간단한 테스트
test_response = llm.invoke("안녕하세요!")
print(f"\n🧪 테스트 응답: {test_response.content[:100]}...")

## 2. 직렬화 가능성 확인 (`is_lc_serializable`)

LangChain의 `is_lc_serializable` 함수를 사용하여 객체가 직렬화 가능한지 확인할 수 있습니다.

### 직렬화란?
- **직렬화(Serialization)**: 객체를 저장이나 전송 가능한 형태로 변환
- **역직렬화(Deserialization)**: 저장된 데이터를 다시 객체로 복원

### LangChain 직렬화 지원
- LangChain 컴포넌트들은 대부분 직렬화를 지원
- `@serializable` 데코레이터로 직렬화 가능 클래스 표시

In [ ]:
############################################################################
# 다양한 객체의 직렬화 가능성 확인
############################################################################

print("🔍 직렬화 가능성 확인")
print("=" * 50)

# 1. LLM 모델 확인
print(f"1. ChatGoogleGenerativeAI: {is_lc_serializable(llm)}")

# 2. 프롬프트 템플릿 확인
prompt = ChatPromptTemplate.from_template("다음 질문에 답해주세요: {question}")
print(f"2. ChatPromptTemplate: {is_lc_serializable(prompt)}")

# 3. 출력 파서 확인
parser = StrOutputParser()
print(f"3. StrOutputParser: {is_lc_serializable(parser)}")

# 4. 체인 확인 (LangChain Expression Language - LCEL)
chain = prompt | llm | parser
print(f"4. Chain (prompt | llm | parser): {is_lc_serializable(chain)}")

# 5. 개별 메시지 확인
human_msg = HumanMessage(content="안녕하세요")
system_msg = SystemMessage(content="당신은 도움이 되는 AI입니다")
print(f"5. HumanMessage: {is_lc_serializable(human_msg)}")
print(f"6. SystemMessage: {is_lc_serializable(system_msg)}")

# 6. 일반 Python 객체 확인
regular_dict = {"key": "value"}
print(f"7. Regular Dict: {is_lc_serializable(regular_dict)}")

print("\n✅ 직렬화 가능성 확인 완료!")
print("📝 True: LangChain 직렬화 지원, False: 지원하지 않음")

## 3. JSON 직렬화 (`dumps`, `dumpd`)

LangChain에서 제공하는 JSON 직렬화 함수들을 사용해봅시다.

### 함수별 특징
- **`dumps()`**: 객체를 JSON 문자열로 직렬화
- **`dumpd()`**: 객체를 Python 딕셔너리로 직렬화
- **`loads()`**: 직렬화된 데이터를 다시 객체로 역직렬화

### 장점
- **가독성**: JSON 형태로 사람이 읽기 쉬움
- **호환성**: 다양한 언어와 플랫폼에서 지원
- **디버깅**: 구조를 쉽게 파악 가능

### 단점
- **제한성**: LangChain 지원 객체만 직렬화 가능
- **크기**: 바이너리 형식보다 용량이 큼

In [ ]:
############################################################################
# JSON 직렬화 실습
############################################################################

print("📦 JSON 직렬화 실습")
print("=" * 50)

# 1. 프롬프트 템플릿 직렬화
print("1. 프롬프트 템플릿 직렬화")
print("-" * 30)

# dumps() - JSON 문자열로 변환
prompt_json_str = dumps(prompt)
print(f"✅ dumps() 성공! 길이: {len(prompt_json_str)} 문자")
print(f"📄 JSON 미리보기: {prompt_json_str[:100]}...")

# dumpd() - Python 딕셔너리로 변환
prompt_dict = dumpd(prompt)
print(f"\n✅ dumpd() 성공! 키 개수: {len(prompt_dict)} 개")
print(f"🔑 딕셔너리 키들: {list(prompt_dict.keys())}")

# 2. 체인 직렬화
print("\n\n2. 체인 직렬화")
print("-" * 30)

try:
    chain_json_str = dumps(chain)
    print(f"✅ 체인 dumps() 성공! 길이: {len(chain_json_str)} 문자")
    
    chain_dict = dumpd(chain)
    print(f"✅ 체인 dumpd() 성공! 키 개수: {len(chain_dict)} 개")
    print(f"🔑 체인 딕셔너리 키들: {list(chain_dict.keys())}")
    
except Exception as e:
    print(f"❌ 체인 직렬화 실패: {e}")

# 3. 개별 컴포넌트 확인
print("\n\n3. 개별 컴포넌트 직렬화")
print("-" * 30)

components = {
    "프롬프트": prompt,
    "파서": parser
}

for name, component in components.items():
    try:
        json_str = dumps(component)
        print(f"✅ {name}: 성공 ({len(json_str)} 문자)")
    except Exception as e:
        print(f"❌ {name}: 실패 - {e}")

print("\n✅ JSON 직렬화 실습 완료!")

In [ ]:
############################################################################
# JSON 역직렬화 실습
############################################################################

print("🔄 JSON 역직렬화 실습")
print("=" * 50)

# 1. 프롬프트 템플릿 복원
print("1. 프롬프트 템플릿 복원")
print("-" * 30)

# JSON 문자열에서 복원
restored_prompt = loads(prompt_json_str)
print(f"✅ 프롬프트 복원 성공!")
print(f"📝 원본 타입: {type(prompt)}")
print(f"📝 복원 타입: {type(restored_prompt)}")
print(f"🔍 동일성 확인: {type(prompt) == type(restored_prompt)}")

# 2. 복원된 프롬프트 테스트
print("\n2. 복원된 프롬프트 테스트")
print("-" * 30)

test_question = "LangChain이란 무엇인가요?"

# 원본 프롬프트
original_messages = prompt.format_messages(question=test_question)
print(f"📤 원본 프롬프트: {original_messages[0].content}")

# 복원된 프롬프트
restored_messages = restored_prompt.format_messages(question=test_question)
print(f"📥 복원 프롬프트: {restored_messages[0].content}")

# 동일성 확인
print(f"🔍 메시지 동일성: {original_messages[0].content == restored_messages[0].content}")

# 3. 실제 LLM 호출 테스트
print("\n3. 복원된 프롬프트로 LLM 호출")
print("-" * 30)

response = llm.invoke(restored_messages)
print(f"💬 응답: {response.content[:100]}...")

print("\n✅ JSON 역직렬화 실습 완료!")

## 4. Pickle 직렬화

Pickle은 Python의 표준 직렬화 라이브러리로, 거의 모든 Python 객체를 직렬화할 수 있습니다.

### Pickle의 특징

**장점:**
- **완전성**: 거의 모든 Python 객체 지원
- **효율성**: 바이너리 형식으로 크기가 작음
- **속도**: JSON보다 빠른 직렬화/역직렬화

**단점:**
- **보안 위험**: 악의적인 코드 실행 가능
- **Python 전용**: 다른 언어에서 읽을 수 없음
- **버전 의존성**: Python 버전별 호환성 문제 가능

### 사용 사례
- **개발 환경**: 복잡한 객체의 임시 저장
- **캐싱**: 연산 결과의 빠른 저장/로드
- **모델 백업**: 전체 모델 상태 보존

In [ ]:
############################################################################
# Pickle 직렬화 실습
############################################################################

print("🥒 Pickle 직렬화 실습")
print("=" * 50)

# 1. 단일 객체 Pickle 저장
print("1. 단일 객체 Pickle 저장")
print("-" * 30)

# 프롬프트 템플릿을 pickle로 저장
prompt_file = "prompt_template.pkl"
with open(prompt_file, 'wb') as f:
    pickle.dump(prompt, f)

file_size = os.path.getsize(prompt_file)
print(f"✅ 프롬프트 템플릿 저장 완료!")
print(f"📁 파일명: {prompt_file}")
print(f"📏 파일 크기: {file_size} bytes")

# 2. 복합 객체 Pickle 저장
print("\n2. 복합 객체 Pickle 저장")
print("-" * 30)

# 여러 객체를 딕셔너리로 묶어서 저장 (LLM 제외)
# LLM 객체는 gRPC 연결 때문에 pickle로 직렬화할 수 없음
model_components_safe = {
    "prompt": prompt,
    "parser": parser,
    "chain_config": dumpd(chain),  # 체인을 JSON 형태로 저장
    "llm_config": {
        "model": llm.model,
        "temperature": llm.temperature,
        "max_output_tokens": llm.max_output_tokens
    },
    "metadata": {
        "created_at": "2025-09-05",
        "model_name": "gemini-1.5-flash",
        "version": "1.0"
    }
}

print("📝 저장 가능한 컴포넌트 확인:")
for name, component in model_components_safe.items():
    if name in ["metadata", "llm_config"]:
        print(f"   ✅ {name}: 일반 딕셔너리 (저장 가능)")
    elif name == "chain_config":
        print(f"   ✅ {name}: JSON 직렬화된 체인 (저장 가능)")
    else:
        try:
            # 테스트로 pickle 직렬화 시도
            pickle.dumps(component)
            print(f"   ✅ {name}: Pickle 직렬화 가능")
        except Exception as e:
            print(f"   ❌ {name}: Pickle 직렬화 불가 - {str(e)[:50]}...")

components_file = "model_components.pkl"
with open(components_file, 'wb') as f:
    pickle.dump(model_components_safe, f)

components_size = os.path.getsize(components_file)
print(f"\n✅ 모델 컴포넌트 저장 완료!")
print(f"📁 파일명: {components_file}")
print(f"📏 파일 크기: {components_size} bytes")
print(f"🔧 저장된 컴포넌트: {list(model_components_safe.keys())}")

print("\n⚠️  참고: LLM 객체는 연결 정보 때문에 pickle로 저장할 수 없어")
print("   설정 정보만 저장하고 나중에 새로 생성해야 합니다.")

print("\n✅ Pickle 저장 실습 완료!")

In [ ]:
############################################################################
# Pickle 역직렬화 실습
############################################################################

print("🔄 Pickle 역직렬화 실습")
print("=" * 50)

# 1. 단일 객체 불러오기
print("1. 단일 객체 불러오기")
print("-" * 30)

# 프롬프트 템플릿 불러오기
with open(prompt_file, 'rb') as f:
    loaded_prompt = pickle.load(f)

print(f"✅ 프롬프트 템플릿 로드 완료!")
print(f"📝 원본 타입: {type(prompt)}")
print(f"📝 로드 타입: {type(loaded_prompt)}")
print(f"🔍 타입 동일성: {type(prompt) == type(loaded_prompt)}")

# 2. 복합 객체 불러오기
print("\n2. 복합 객체 불러오기")
print("-" * 30)

with open(components_file, 'rb') as f:
    loaded_components = pickle.load(f)

print(f"✅ 모델 컴포넌트 로드 완료!")
print(f"🔧 로드된 컴포넌트: {list(loaded_components.keys())}")
print(f"📊 메타데이터: {loaded_components['metadata']}")

# 3. 로드된 객체들 테스트
print("\n3. 로드된 객체들 테스트")
print("-" * 30)

# 로드된 프롬프트 테스트
test_question = "Python에서 Pickle의 장단점은?"
loaded_messages = loaded_prompt.format_messages(question=test_question)
print(f"📤 로드된 프롬프트: {loaded_messages[0].content}")

# 로드된 LLM으로 응답 생성
if 'llm_config' in loaded_components:
    llm_config = loaded_components['llm_config']
    print(f"🔧 LLM 설정 로드: {llm_config}")
    
    # 새로운 LLM 객체 생성
    recreated_llm = ChatGoogleGenerativeAI(
        model=llm_config['model'],
        temperature=llm_config['temperature'],
        max_output_tokens=llm_config['max_output_tokens']
    )
    
    response = recreated_llm.invoke(loaded_messages)
    print(f"💬 재생성된 LLM 응답: {response.content[:100]}...")
else:
    print("⚠️  LLM 설정을 찾을 수 없습니다. 기존 LLM 사용...")
    response = llm.invoke(loaded_messages)
    print(f"💬 기존 LLM 응답: {response.content[:100]}...")

# 로드된 체인 설정에서 체인 복원
if 'chain_config' in loaded_components:
    try:
        # JSON 직렬화된 체인 설정을 복원
        chain_config = loaded_components['chain_config']
        restored_chain = loads(json.dumps(chain_config))
        
        chain_response = restored_chain.invoke({"question": test_question})
        print(f"🔗 복원된 체인 응답: {chain_response[:100]}...")
    except Exception as e:
        print(f"⚠️  체인 복원 에러: {e}")
        print("   기존 체인으로 테스트...")
        chain_response = chain.invoke({"question": test_question})
        print(f"🔗 기존 체인 응답: {chain_response[:100]}...")
else:
    print("⚠️  체인 설정을 찾을 수 없습니다.")

print("\n✅ Pickle 역직렬화 실습 완료!")

## 5. 모델 저장 및 불러오기 실전 예제

실제 애플리케이션에서 사용할 수 있는 모델 저장/불러오기 패턴을 구현해봅시다.

### 실전 시나리오
1. **설정 저장**: 모델 파라미터와 프롬프트를 JSON으로 저장
2. **모델 백업**: 전체 모델을 Pickle로 백업
3. **버전 관리**: 다양한 버전의 모델 관리
4. **안전한 로딩**: 에러 처리와 검증 포함

In [ ]:
############################################################################
# 모델 관리 클래스 구현
############################################################################

import datetime
from pathlib import Path

class ModelManager:
    """LangChain 모델의 저장과 불러오기를 관리하는 클래스"""
    
    def __init__(self, base_dir="saved_models"):
        self.base_dir = Path(base_dir)
        self.base_dir.mkdir(exist_ok=True)
    
    def save_model_config(self, model_name, components):
        """JSON 형태로 모델 설정 저장"""
        config_data = {
            "metadata": {
                "name": model_name,
                "created_at": datetime.datetime.now().isoformat(),
                "langchain_version": "latest"
            }
        }
        
        # 직렬화 가능한 컴포넌트들만 저장
        for name, component in components.items():
            if is_lc_serializable(component):
                try:
                    config_data[name] = dumpd(component)
                    print(f"✅ {name}: JSON 저장 성공")
                except Exception as e:
                    print(f"❌ {name}: JSON 저장 실패 - {e}")
            else:
                print(f"⚠️  {name}: 직렬화 불가능 (JSON 저장 제외)")
        
        # JSON 파일 저장
        config_file = self.base_dir / f"{model_name}_config.json"
        with open(config_file, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=2, ensure_ascii=False)
        
        print(f"📁 설정 파일 저장: {config_file}")
        return config_file
    
    def save_model_pickle(self, model_name, components):
        """Pickle 형태로 전체 모델 저장"""
        # LLM 객체는 pickle로 저장할 수 없으므로 안전한 형태로 변환
        safe_components = {}
        
        for name, component in components.items():
            if name == "llm":
                # LLM 객체는 설정 정보만 저장
                safe_components["llm_config"] = {
                    "model": component.model,
                    "temperature": component.temperature,
                    "max_output_tokens": component.max_output_tokens
                }
                print(f"⚠️  {name}: LLM 객체를 설정 정보로 변환하여 저장")
            elif name == "chain":
                # 체인은 JSON 직렬화하여 저장
                try:
                    safe_components["chain_config"] = dumpd(component)
                    print(f"✅ {name}: 체인을 JSON 설정으로 변환하여 저장")
                except Exception as e:
                    print(f"❌ {name}: 체인 직렬화 실패 - {e}")
            else:
                # 다른 컴포넌트들은 pickle 가능성 확인 후 저장
                try:
                    pickle.dumps(component)  # 테스트
                    safe_components[name] = component
                    print(f"✅ {name}: Pickle 직렬화 가능")
                except Exception as e:
                    print(f"❌ {name}: Pickle 직렬화 불가 - {str(e)[:50]}...")
        
        pickle_data = {
            "metadata": {
                "name": model_name,
                "created_at": datetime.datetime.now().isoformat(),
                "python_version": f"{os.sys.version_info.major}.{os.sys.version_info.minor}"
            },
            "components": safe_components
        }
        
        pickle_file = self.base_dir / f"{model_name}.pkl"
        with open(pickle_file, 'wb') as f:
            pickle.dump(pickle_data, f)
        
        print(f"📁 Pickle 파일 저장: {pickle_file}")
        return pickle_file
    
    def load_model_config(self, model_name):
        """JSON 설정에서 모델 불러오기"""
        config_file = self.base_dir / f"{model_name}_config.json"
        
        if not config_file.exists():
            raise FileNotFoundError(f"설정 파일을 찾을 수 없습니다: {config_file}")
        
        with open(config_file, 'r', encoding='utf-8') as f:
            config_data = json.load(f)
        
        print(f"📖 설정 파일 로드: {config_file}")
        print(f"📊 메타데이터: {config_data['metadata']}")
        
        # 컴포넌트 복원
        components = {}
        for name, component_data in config_data.items():
            if name != "metadata":
                try:
                    components[name] = loads(json.dumps(component_data))
                    print(f"✅ {name}: JSON 복원 성공")
                except Exception as e:
                    print(f"❌ {name}: JSON 복원 실패 - {e}")
        
        return components, config_data['metadata']
    
    def load_model_pickle(self, model_name):
        """Pickle 파일에서 모델 불러오기"""
        pickle_file = self.base_dir / f"{model_name}.pkl"
        
        if not pickle_file.exists():
            raise FileNotFoundError(f"Pickle 파일을 찾을 수 없습니다: {pickle_file}")
        
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
        
        print(f"📖 Pickle 파일 로드: {pickle_file}")
        print(f"📊 메타데이터: {pickle_data['metadata']}")
        
        components = pickle_data['components']
        
        # LLM 설정이 있다면 새로운 LLM 객체 생성
        if 'llm_config' in components:
            try:
                llm_config = components['llm_config']
                components['llm'] = ChatGoogleGenerativeAI(
                    model=llm_config['model'],
                    temperature=llm_config['temperature'],
                    max_output_tokens=llm_config['max_output_tokens']
                )
                print("✅ LLM 객체 재생성 완료")
            except Exception as e:
                print(f"❌ LLM 객체 재생성 실패: {e}")
        
        # 체인 설정이 있다면 체인 복원
        if 'chain_config' in components:
            try:
                chain_config = components['chain_config']
                components['chain'] = loads(json.dumps(chain_config))
                print("✅ 체인 객체 복원 완료")
            except Exception as e:
                print(f"❌ 체인 객체 복원 실패: {e}")
        
        return components, pickle_data['metadata']
    
    def list_saved_models(self):
        """저장된 모델 목록 반환"""
        models = []
        for file in self.base_dir.glob("*"):
            if file.suffix == ".json" and "_config" in file.name:
                model_name = file.name.replace("_config.json", "")
                models.append({
                    "name": model_name,
                    "config_file": file,
                    "pickle_file": self.base_dir / f"{model_name}.pkl"
                })
        return models

# ModelManager 인스턴스 생성
model_manager = ModelManager()
print("🎯 ModelManager 초기화 완료!")

In [ ]:
############################################################################
# 실전 예제 실행
############################################################################

print("🚀 실전 모델 저장/불러오기 예제")
print("=" * 50)

# 1. 모델 저장
print("1. 모델 저장하기")
print("-" * 30)

# 저장할 컴포넌트들 준비
my_components = {
    "llm": llm,
    "prompt": prompt,
    "parser": parser,
    "chain": chain
}

model_name = "gemini_qa_model_v1"

# JSON 설정 저장
print("🔧 JSON 설정 저장 중...")
config_file = model_manager.save_model_config(model_name, my_components)

# Pickle 전체 저장
print("\n🥒 Pickle 전체 저장 중...")
pickle_file = model_manager.save_model_pickle(model_name, my_components)

# 2. 저장된 모델 목록 확인
print("\n\n2. 저장된 모델 목록")
print("-" * 30)

saved_models = model_manager.list_saved_models()
for model_info in saved_models:
    config_exists = model_info["config_file"].exists()
    pickle_exists = model_info["pickle_file"].exists()
    
    print(f"📋 모델명: {model_info['name']}")
    print(f"   ✅ 설정 파일: {'있음' if config_exists else '없음'}")
    print(f"   ✅ Pickle 파일: {'있음' if pickle_exists else '없음'}")

print("\n✅ 모델 저장 완료!")

In [ ]:
############################################################################
# 모델 불러오기 및 테스트
############################################################################

print("📂 모델 불러오기 및 테스트")
print("=" * 50)

# 1. JSON 설정에서 불러오기
print("1. JSON 설정에서 모델 불러오기")
print("-" * 30)

try:
    json_components, json_metadata = model_manager.load_model_config(model_name)
    print(f"✅ JSON 불러오기 성공!")
    print(f"📊 불러온 컴포넌트: {list(json_components.keys())}")
except Exception as e:
    print(f"❌ JSON 불러오기 실패: {e}")
    json_components = None

# 2. Pickle에서 불러오기
print("\n2. Pickle에서 모델 불러오기")
print("-" * 30)

try:
    pickle_components, pickle_metadata = model_manager.load_model_pickle(model_name)
    print(f"✅ Pickle 불러오기 성공!")
    print(f"📊 불러온 컴포넌트: {list(pickle_components.keys())}")
except Exception as e:
    print(f"❌ Pickle 불러오기 실패: {e}")
    pickle_components = None

# 3. 불러온 모델들 성능 테스트
print("\n3. 불러온 모델들 성능 테스트")
print("-" * 30)

test_question = "LangChain 모델 직렬화의 장점은 무엇인가요?"
print(f"🧪 테스트 질문: {test_question}")

# JSON 모델 테스트
if json_components and "prompt" in json_components:
    try:
        json_prompt = json_components["prompt"]
        json_messages = json_prompt.format_messages(question=test_question)
        json_response = llm.invoke(json_messages)
        print(f"\n📄 JSON 모델 응답: {json_response.content[:100]}...")
    except Exception as e:
        print(f"❌ JSON 모델 테스트 실패: {e}")

# Pickle 모델 테스트
if pickle_components:
    try:
        pickle_chain = pickle_components["chain"]
        pickle_response = pickle_chain.invoke({"question": test_question})
        print(f"\n🥒 Pickle 모델 응답: {pickle_response[:100]}...")
    except Exception as e:
        print(f"❌ Pickle 모델 테스트 실패: {e}")

print("\n✅ 모델 불러오기 및 테스트 완료!")

## 6. 체인 직렬화 실습

복잡한 LangChain 체인의 직렬화를 연습해봅시다.

### 체인 직렬화의 중요성
- **재사용성**: 복잡한 체인 구성을 재사용
- **배포**: 프로덕션 환경에 체인 배포
- **버전 관리**: 체인의 다양한 버전 관리
- **협업**: 팀원 간 체인 공유

### 직렬화 가능한 체인 vs 불가능한 체인
- ✅ **가능**: LangChain 기본 컴포넌트로만 구성된 체인
- ❌ **불가능**: 사용자 정의 함수나 람다가 포함된 체인

In [ ]:
############################################################################
# 복잡한 체인 생성 및 직렬화
# 💡LangChain에서 직렬화 가능한 체인을 만들 때의 핵심 원칙:
#   - 가능한 한 LangChain의 기본 컴포넌트만 사용
#   - 커스텀 함수나 람다 함수는 직렬화되지 않으므로 피하기
#   - 프롬프트 템플릿에 고정 값들을 직접 포함시키기
############################################################################

from langchain_core.runnables import RunnablePassthrough, RunnableParallel

print("🔗 복잡한 체인 직렬화 실습")
print("=" * 50)

# 1. 다양한 타입의 체인 생성
print("1. 다양한 타입의 체인 생성")
print("-" * 30)

# 기본 순차 체인
basic_chain = prompt | llm | parser
print(f"✅ 기본 체인: {is_lc_serializable(basic_chain)}")

# 병렬 체인
summary_prompt = ChatPromptTemplate.from_template("다음 텍스트를 요약해주세요: {text}")
analysis_prompt = ChatPromptTemplate.from_template("다음 텍스트의 주요 키워드를 추출해주세요: {text}")

parallel_chain = RunnableParallel({
    "summary": summary_prompt | llm | parser,
    "keywords": analysis_prompt | llm | parser
})
print(f"✅ 병렬 체인: {is_lc_serializable(parallel_chain)}")

# 조건부 체인 (더 간단한 접근법)
conditional_prompt = ChatPromptTemplate.from_template(
    "질문: {question}\n\n답변해주세요:"
)

# 단순한 체인으로 구성 (직렬화 가능)
conditional_chain = conditional_prompt | llm | parser
print(f"✅ 조건부 체인: {is_lc_serializable(conditional_chain)}")

# 또는 질문 타입을 포함한 다른 접근법
enhanced_prompt = ChatPromptTemplate.from_template(
    "다음은 일반적인 질문입니다.\n질문: {question}\n\n답변해주세요:"
)

enhanced_chain = enhanced_prompt | llm | parser
print(f"✅ 향상된 체인: {is_lc_serializable(enhanced_chain)}")

# 2. 체인들의 직렬화 테스트
print("\n2. 체인들의 직렬화 테스트")
print("-" * 30)

chains_to_test = {
    "basic_chain": basic_chain,
    "parallel_chain": parallel_chain,
    "conditional_chain": conditional_chain,
    "enhanced_chain": enhanced_chain
}

serialized_chains = {}

for name, chain in chains_to_test.items():
    try:
        # JSON 직렬화 시도
        chain_json = dumps(chain)
        serialized_chains[name] = chain_json
        print(f"✅ {name}: JSON 직렬화 성공 ({len(chain_json)} 문자)")
        
        # 역직렬화 테스트
        restored_chain = loads(chain_json)
        print(f"   ↳ 역직렬화: ✅ 성공")
        
    except Exception as e:
        print(f"❌ {name}: 직렬화 실패 - {e}")

print("\n✅ 체인 직렬화 테스트 완료!")

In [ ]:
############################################################################
# 직렬화된 체인 실행 테스트
############################################################################

print("🧪 직렬화된 체인 실행 테스트")
print("=" * 50)

# 테스트 데이터
test_inputs = {
    "basic_chain": {"question": "LangChain이란 무엇인가요?"},
    "parallel_chain": {"text": "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다."},
    "conditional_chain": {"question": "Python의 장점은 무엇인가요?"},
    "enhanced_chain": {"question": "AI의 미래는 어떨까요?"}
}

# 각 체인 실행 테스트
for chain_name in serialized_chains.keys():
    if chain_name in test_inputs:
        print(f"\n🔗 {chain_name} 테스트")
        print("-" * 25)
        
        try:
            # 원본 체인 실행
            original_chain = chains_to_test[chain_name]
            original_result = original_chain.invoke(test_inputs[chain_name])
            
            # 직렬화된 체인 복원 및 실행
            restored_chain = loads(serialized_chains[chain_name])
            restored_result = restored_chain.invoke(test_inputs[chain_name])
            
            print(f"📤 입력: {test_inputs[chain_name]}")
            
            if isinstance(original_result, dict):
                print(f"📥 원본 결과: {str(original_result)[:100]}...")
                print(f"🔄 복원 결과: {str(restored_result)[:100]}...")
            else:
                print(f"📥 원본 결과: {original_result[:100]}...")
                print(f"🔄 복원 결과: {restored_result[:100]}...")
                
            print("✅ 실행 성공!")
            
        except Exception as e:
            print(f"❌ 실행 실패: {e}")

print("\n\n🎯 체인 직렬화 실습 완료!")

## 정리 및 모범 사례

이 노트북에서 배운 LangChain 모델 직렬화 방법들을 정리해봅시다.

### 🎯 주요 학습 내용

1. **직렬화 가능성 확인**: `is_lc_serializable()` 함수 활용
2. **JSON 직렬화**: `dumps()`, `dumpd()`, `loads()` 함수 사용
3. **Pickle 직렬화**: Python의 `pickle` 모듈 활용
4. **모델 관리**: 체계적인 저장/불러오기 시스템 구축
5. **체인 직렬화**: 복잡한 체인의 저장과 복원

### 📋 직렬화 방법 선택 가이드

| 상황 | 추천 방법 | 이유 |
|------|-----------|------|
| **설정 공유** | JSON (`dumps`) | 가독성, 플랫폼 독립성 |
| **프로덕션 배포** | JSON + 검증 | 안정성, 버전 관리 |
| **개발/테스트** | Pickle | 완전성, 편의성 |
| **팀 협업** | JSON | 호환성, 디버깅 용이 |
| **모델 백업** | Pickle | 전체 상태 보존 |

### ⚠️ 주의사항

1. **보안**: Pickle 파일은 신뢰할 수 있는 소스에서만 로드
2. **버전 호환성**: LangChain 버전 변경 시 직렬화 호환성 확인
3. **API 키**: 직렬화 시 민감한 정보 제외
4. **파일 관리**: 정기적인 정리로 디스크 공간 관리
5. **테스트**: 직렬화/역직렬화 후 항상 동작 검증

### 🚀 실전 활용 팁

1. **버전 태깅**: 모델 파일에 버전 정보 포함
2. **메타데이터**: 생성 일시, 환경 정보 저장
3. **에러 처리**: 로딩 실패 시 대체 방안 준비
4. **성능 모니터링**: 직렬화된 모델의 성능 추적
5. **백업 전략**: 중요한 모델의 다중 백업 유지

In [ ]:
############################################################################
# 생성된 파일들 정리 (선택사항)
############################################################################

print("🧹 생성된 파일들 정리")
print("=" * 30)

# 정리할 파일들 목록
files_to_clean = [
    "prompt_template.pkl",
    "model_components.pkl"
]

# 생성된 모델 디렉토리도 포함
if model_manager.base_dir.exists():
    for file in model_manager.base_dir.glob("*"):
        files_to_clean.append(str(file))

print("📁 정리 대상 파일들:")
for file_path in files_to_clean:
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        print(f"   {file_path} ({file_size} bytes)")
    else:
        print(f"   {file_path} (존재하지 않음)")

# 파일 삭제 여부 선택
print(f"\n총 {len([f for f in files_to_clean if os.path.exists(f)])} 개의 파일이 정리 대상입니다.")
print("파일들을 삭제하려면 아래 줄의 주석을 해제하세요:")
print("# cleanup_files = True")

cleanup_files = False  # True로 변경하면 파일 삭제

if cleanup_files:
    deleted_count = 0
    for file_path in files_to_clean:
        try:
            if os.path.exists(file_path):
                if os.path.isfile(file_path):
                    os.remove(file_path)
                    print(f"🗑️  삭제됨: {file_path}")
                    deleted_count += 1
        except Exception as e:
            print(f"❌ 삭제 실패: {file_path} - {e}")
    
    # 빈 디렉토리 삭제
    try:
        if model_manager.base_dir.exists() and not any(model_manager.base_dir.iterdir()):
            model_manager.base_dir.rmdir()
            print(f"🗑️  빈 디렉토리 삭제: {model_manager.base_dir}")
    except:
        pass
        
    print(f"\n✅ 총 {deleted_count}개 파일이 정리되었습니다.")
else:
    print("\n💡 정리를 원하면 위의 cleanup_files 변수를 True로 설정하세요.")

print("\n🎉 LangChain 모델 직렬화 실습이 완료되었습니다!")
print("📚 배운 내용을 실제 프로젝트에 적용해보세요!")